In [1]:

# tools/kernel_concurrency_test.py
# Usage: python3 tools/kernel_concurrency_test.py
# Make sure you run this in the Python environment that has CuPy and your repo available.

import time
import math
import numpy as np
import cupy as cp
import sys
sys.path.append('/mnt/d/packing/code/core/')
import pack_cuda

def make_input(N):
    # small random poses in sensible range
    rng = np.random.RandomState(123)
    x = rng.uniform(-5.0, 5.0, size=N)
    y = rng.uniform(-5.0, 5.0, size=N)
    t = rng.uniform(-math.pi, math.pi, size=N)
    xyt = np.stack([x, y, t], axis=1).astype(np.float64)
    return xyt

def run_trial(num_streams, iters_per_stream, xyt_np):
    pack_cuda._ensure_initialized()  # make sure module is initialized
    n = xyt_np.shape[0]
    # Flatten 3xN row-major as kernel expects
    xyt_3xN = cp.ascontiguousarray(cp.asarray(xyt_np).T).ravel()

    # Grab raw kernel and device arrays from pack_cuda (uses private names)
    kernel = pack_cuda._overlap_list_total_kernel
    piece_xy = pack_cuda._piece_xy_d
    piece_nverts = pack_cuda._piece_nverts_d
    num_pieces = np.int32(pack_cuda._num_pieces)

    # Precreate per-stream outputs
    streams = [cp.cuda.Stream(non_blocking=True) for _ in range(num_streams)]
    out_totals = [cp.zeros(1, dtype=cp.float64) for _ in range(num_streams)]

    # Warmup single call per stream to get JIT/compilation out of the way
    for s_idx, stream in enumerate(streams):
        with stream:
            kernel(
                (1,), (n,),
                (xyt_3xN, np.int32(n), piece_xy, piece_nverts, num_pieces, out_totals[s_idx], cp.zeros(1)),
                stream=stream
            )
    # Ensure warmup finished
    for s in streams:
        s.synchronize()

    # Timed run: launch iters_per_stream kernels on each stream (back-to-back)
    start = time.time()
    for k in range(iters_per_stream):
        for s_idx, stream in enumerate(streams):
            with stream:
                kernel(
                    (1,), (n,),
                    (xyt_3xN, np.int32(n), piece_xy, piece_nverts, num_pieces, out_totals[s_idx], cp.zeros(1)),
                    stream=stream
                )
    # Wait for all streams to finish
    for s in streams:
        s.synchronize()
    end = time.time()

    elapsed = end - start
    total_kernels = num_streams * iters_per_stream
    kernels_per_sec = total_kernels / elapsed
    return kernels_per_sec, elapsed


# Parameters to tune
N = 150                       # number of trees (threads per block)
iters = 20                    # iterations per stream
xyt = make_input(N)

packs = [1, 2, 4, 8, 16, 32]  # number of concurrent streams to test
print("Streams\tKernels/sec\tElapsed(s)")
for s in packs:
    kps, t = run_trial(s, iters, xyt)
    print(f"{s}\t{int(kps)}\t\t{t:.3f}")

local
Streams	Kernels/sec	Elapsed(s)


: 

In [ ]:
2

2